# Create Data

This is the completed solution for the [Create Data Tutorial](https://developers.arcgis.com/labs/data/python/create-data/). 

[ArcGIS tutorials](https://developers.arcgis.com/labs/) are short guides demonstrating the three phases of building geospatial apps: Data, Design, Develop.

In [1]:
import getpass
from arcgis.gis import GIS

password = getpass.getpass("Please enter password: ")
gis = GIS('https://www.arcgis.com', 'username', password)

Please enter password: ········


In [2]:
'''
You can also query the Parks and Open Space Feature Layer that
you created in the Import Data Tutorial:

  `query = 'title: "Parks*" AND type: "Feature Service"'
  
  Search for items outside your organization by setting the 
  keyword `outside_org` (default: `False`) to `True`:
  
  `search_results = gis.content.search(query=query, max_itmes=10, outside_org=True)`
'''

query = 'title: "Trailheads*" AND type: "Feature Service"'
search_results = gis.content.search(query=query, max_items=10)
search_results

[<Item title:"Trailheads" type:Feature Layer Collection owner:cff_dev>]

In [3]:
'''
If `csv_data.url` returns the empty string, make sure
that you've made the `Trailheads` portal item public
'''

csv_data = search_results[0]
csv_data.url

'https://services9.arcgis.com/ZPYuLYCJmpcXcod0/arcgis/rest/services/Trailheads/FeatureServer'

In [4]:
trailheads_layer = csv_data.layers[0]

In [5]:
trailheads_layer.properties.name

'Trailheads'

In [8]:
for field in trailheads_layer.properties['fields']:
    print('Name: {:16s}\tType: {}'.format(field['name'], field['actualType']))

Name: OBJECTID        	Type: nvarchar
Name: TRL_NAME        	Type: nvarchar
Name: PARK_NAME       	Type: nvarchar
Name: TRL_ID          	Type: int
Name: SEG_ID          	Type: nvarchar
Name: FEATURE         	Type: nvarchar
Name: FEAT_CMNT       	Type: nvarchar
Name: ELEV_FT         	Type: nvarchar
Name: CITY_JUR        	Type: nvarchar
Name: ZIP_CODE        	Type: nvarchar
Name: SUP_DIST        	Type: nvarchar
Name: X_STREET        	Type: nvarchar
Name: PARKING         	Type: nvarchar
Name: PHOTO           	Type: nvarchar
Name: LAT             	Type: float
Name: LON             	Type: float
Name: FID             	Type: nvarchar
Name: x               	Type: float
Name: y               	Type: float
Name: ObjectId2       	Type: int


In [9]:
### Add Layer to Map

In [17]:
map = gis.map()

# For the ArcGIS API for JavaScript, we would order coordinates as [`longitude`, `latitude`]
coords = [ -118.71511, 34.09042 ]

# But for the ArcGIS API for Python, order coordinates as [`latitude`, `longitude`]
coords.reverse()

In [20]:
### Create Features

In [26]:
from arcgis import geometry
from arcgis import features


def create_feature(map, location):
    try:
        '''
        `location` will be a dict the following format:
        
        ```json
        {
            'spatialReference': 
            {
                'latestWkid': 3857, 
                'wkid': 102100
            }, 
            'x': -13228260.775215352, 
            'y': 4044978.1647766223
        }
        ```
        '''
        
        object_id = 1
        point = geometry.Point(location)
        
        feature = features.Feature(
            geometry=point, 
            attributes={
                'OBJECTID': object_id,
                'PARK_NAME': 'My Park',
                'TRL_NAME': 'Foobar Trail',
                'ELEV_FT': '5000'
            }
        )
        
        trailheads_layer.edit_features(adds=[feature])
        map.draw(point)
        
    except Exception as e:
        print("Couldn't create the feature. {}".format(str(e)))

In [27]:
### Map of LA

In [28]:
map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True, zoom=9.0)

In [29]:
map.basemap = 'topo-vector'
map.center = coords
map.zoom = 11

In [30]:
map.on_click(create_feature)

### Load Trailheads Layer

Now you can clear the map, and load the Trailheads layer with the points you just added to the layer!

In [32]:
map.clear_graphics()

In [31]:
map.add_layer(trailheads_layer)